prepare BERT model

In [28]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

use BERT to embed pile dataset

In [34]:
inputs = tokenizer("Hello world my name is joseph!", return_tensors="pt")
outputs = model(**inputs)

In [2]:
from datasets import load_dataset

ds = load_dataset("JeanKaddour/minipile")

/Users/josephtey/Projects/auto-ed-coder/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import pandas as pd
from datetime import datetime

# Initialize an empty DataFrame
df_all_sentences = pd.DataFrame(columns=['sentence'])

row_index = 0
for text in ds['validation']['text']:
    sentences = sent_tokenize(text)
    for sentence in sentences:
        # Use .at to set the value in the DataFrame
        df_all_sentences.at[row_index, 'sentence'] = sentence
        row_index += 1

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'files/all_sentences_{timestamp}.csv'
df_all_sentences.to_csv(filename, index=False)


[nltk_data] Downloading package punkt to /Users/josephtey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyboardInterrupt: 

In [43]:
from torch.utils.data import Dataset

class MiniPileDataset(Dataset):
    def __init__(self, sentences, embeddings):
        self.sentences = sentences
        self.embeddings = embeddings

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx], self.embeddings[idx]

In [47]:
# Write the dataset to a CSV file
import csv
from datetime import datetime

# Embed the first 100 sentences
embeddings = []
for i, sent in enumerate(df_all_sentences['sentence'][:100]):
    try:
        inputs = tokenizer(sent, return_tensors="pt")
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze(0).detach().tolist()
        embeddings.append(embedding)
    except Exception as e:
        continue
    print(f"Processed {i+1} sentences")

# Initialize the dataset
sentence_dataset = MiniPileDataset(df_all_sentences['sentence'][:100].tolist(), embeddings)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'files/all_sentences_with_embeddings_{timestamp}.csv'
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['sentence', 'embedding'])
    for sentence, embedding in zip(sentence_dataset.sentences, sentence_dataset.embeddings):
        writer.writerow([sentence, embedding])

# Example usage: get the first sentence and its embedding
first_sentence, first_embedding = sentence_dataset[0]


Processed 1 sentences
Processed 2 sentences
Processed 3 sentences
Error processing sentence: From /var/apt/history.log:
Start-Date: 2017-11-06  01:15:38
Commandline: apt-get autoremove
Requested-By: praveen (1000)
Remove: linux-headers-4.10.0-28-generic:amd64 (4.10.0-28.32~16.04.2), librpmsign3:amd64 (4.12.0.1+dfsg1-3build3), linux-image-extra-4.10.0-33-generic:amd64 (4.10.0-33.37~16.04.1),
librpmbuild3:amd64 (4.12.0.1+dfsg1-3build3),
linux-image-extra-4.10.0-35-generic:amd64 (4.10.0-35.39~16.04.1),
debugedit:amd64 (4.12.0.1+dfsg1-3build3),
linux-headers-4.10.0-33-generic:amd64 (4.10.0-33.37~16.04.1),
linux-image-4.10.0-28-generic:amd64 (4.10.0-28.32~16.04.2),
linux-headers-4.10.0-35-generic:amd64 (4.10.0-35.39~16.04.1),
rpm:amd64 (4.12.0.1+dfsg1-3build3),
linux-image-4.10.0-33-generic:amd64 (4.10.0-33.37~16.04.1),
linux-headers-4.10.0-28:amd64 (4.10.0-28.32~16.04.2),
linux-headers-4.10.0-33:amd64 (4.10.0-33.37~16.04.1),
linux-headers-4.10.0-35:amd64 (4.10.0-35.39~16.04.1),
linux-image

train SAE

In [57]:
import torch.optim as optim
import torch
from utils.sae import SparseAutoencoder, SparseAutoencoderConfig

# Assuming `dataset` is a PyTorch Dataset loaded and ready to use
data_loader = torch.utils.data.DataLoader(sentence_dataset, batch_size=512, shuffle=True)

# Initialize the model
config = SparseAutoencoderConfig(d_model=768, d_sparse=8 * 768)
model = SparseAutoencoder(config)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    total_loss = 0
    for sentence, embeddings in data_loader:
        optimizer.zero_grad()
        
        # Assuming data is already on the correct device and in the correct format
        _, _, loss, _ = model.forward(embeddings, return_loss=True, sparsity_scale=1.0)
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f"Epoch {epoch+1}, Average Loss: {total_loss / len(data_loader)}")

ModuleNotFoundError: No module named 'pydantic'